In [14]:
import pandas as pd
import os

import numpy as np

In [135]:
# Specify the data folder
data_folder = "./../original_data"

# Read each CSV file into a separate DataFrame
attendance_df = pd.read_csv(os.path.join(data_folder, 'attendance.csv'))
games_df = pd.read_csv(os.path.join(data_folder, 'games.csv'))
standings_df = pd.read_csv(os.path.join(data_folder, 'standings.csv'))

# Year filtering

In [136]:
attendance_df = attendance_df[(attendance_df['year'] >= 2009) & (attendance_df['year'] <= 2019)]
games_df = games_df[(games_df['year'] >= 2009) & (games_df['year'] <= 2019)]
standings_df = standings_df[(standings_df['year'] >= 2007) & (standings_df['year'] <= 2019)] # Here we allow two years before

# Splitt regular season and other 

In [137]:
# We create a new df with this values called spetial games
special_games_df = games_df[games_df['week'].isin(['WildCard', 'Division', 'ConfChamp', 'SuperBowl'])].copy()

# We drop the special games from the original df
games_df = games_df[~games_df['week'].isin(['WildCard', 'Division', 'ConfChamp', 'SuperBowl'])].copy()

# Column types


In [138]:
# Specify column types for attendance_df
attendance_df[["year", "week", "weekly_attendance"]] = attendance_df[["year", "week", "weekly_attendance"]].astype("Int16", errors="ignore")
attendance_df["team_name"] = attendance_df["team_name"].astype("string")

# Specify column types for games_df
games_df[["year", "week", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]] = games_df[["year", "week", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]].astype("Int16", errors="ignore")
games_df[["home_team_name", "away_team_name", "home_team_city", "away_team_city"]] = games_df[["home_team_name", "away_team_name", "home_team_city", "away_team_city"]].astype("string")

# Specify column types for standings_df
standings_df[["year", "wins", "loss", "points_for", "points_against", "points_differential"]] = standings_df[["year", "wins", "loss", "points_for", "points_against", "points_differential"]].astype("Int16", errors="ignore")
standings_df[["margin_of_victory", "strength_of_schedule", "simple_rating", "offensive_ranking", "defensive_ranking"]] = standings_df[["margin_of_victory", "strength_of_schedule", "simple_rating", "offensive_ranking", "defensive_ranking"]].astype("float")
standings_df[["playoffs", "team_name", "sb_winner"]] = standings_df[["playoffs", "team_name", "sb_winner"]].astype("string")

# Specify column types for special_games_df
special_games_df[["year", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]] = special_games_df[["year", "pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"]].astype("Int16", errors="ignore")
special_games_df[["week", "home_team_name", "away_team_name", "home_team_city"]] = special_games_df[["week", "home_team_name", "away_team_name", "home_team_city"]].astype("string")

# Create attendance as percentage

In [139]:
# Maximum of all seasons
attendance_df["weekly_attendance_ratio"] = attendance_df.groupby("team_name")["weekly_attendance"].transform(lambda x: x / x.max()) 

# Maximum for each season since they moved  the stadium
selected_teams_names = ["Chargers", "Rams"]
attendance_df_subset = attendance_df[attendance_df["team_name"].isin(selected_teams_names)].copy()
attendance_df_subset["weekly_attendance_ratio"] = attendance_df_subset.groupby(["team_name", "year"])["weekly_attendance"].transform(lambda x: x / x.max())

# Merging 
attendance_df.loc[attendance_df_subset.index, "weekly_attendance_ratio"] = attendance_df_subset["weekly_attendance_ratio"]

# Select relevant columns for attendance_df
attendance_df = attendance_df[["year", "team_name", "week", "weekly_attendance", "weekly_attendance_ratio"]]# Resort columns

In [17]:
# import matplotlib.pyplot as plt

# teams_to_plot = attendance_df["team_full_name"].unique()

# for team in teams_to_plot:
#     team_data = attendance_df[attendance_df["team_full_name"] == team]
#     plt.figure(figsize=(10, 4))
#     plt.plot(
#         pd.to_datetime(team_data["year"].astype(str) + "-W" + team_data["week"].astype(str) + "-1", format="%Y-W%W-%w"),
#         team_data["weekly_attendance"],
#         label=team
#     )
#     plt.xlabel("Date")
#     plt.ylabel("Weekly Attendance")
#     plt.title(f"Weekly Attendance Over Time: {team}")
#     plt.legend()
#     plt.tight_layout()
#     plt.show()


# Merge attendance to games 

In [140]:
games_df = pd.merge(
    games_df,
    attendance_df,
    left_on=['home_team_name', 'year', 'week'],
    right_on=['team_name', 'year', 'week'],
    how='inner'
).drop(columns=['team_name']).sort_values(by=['year', 'week', 'time'])

In [141]:
# Create previous game attendance by grouping by home_team_name and lagging
games_df = games_df.sort_values(by=["home_team_name", "year", "week"])
games_df["prev_game_attendance"] = games_df.groupby("home_team_name")["weekly_attendance"].shift(1)

In [130]:
games_df

,year,week,home_team,away_team,winner,tie,day,date,time,home_team_name,...,turnovers_away,day_sin,day_cos,month,month_sin,month_cos,noon,mid_afternoon,late_afternoon,evening
16,2009,2,San Francisco 49ers,Seattle Seahawks,San Francisco 49ers,NaN,6,September 20,4:05PM,49ers,...,1,-0.781831,0.623490,0,0.000000,1.000000,False,True,False,False
48,2009,4,San Francisco 49ers,St. Louis Rams,San Francisco 49ers,NaN,6,October 4,4:15PM,49ers,...,3,-0.781831,0.623490,1,0.951057,0.309017,False,True,False,False
62,2009,5,San Francisco 49ers,Atlanta Falcons,Atlanta Falcons,NaN,6,October 11,4:05PM,49ers,...,2,-0.781831,0.623490,1,0.951057,0.309017,False,True,False,False
116,2009,9,San Francisco 49ers,Tennessee Titans,Tennessee Titans,NaN,6,November 8,4:15PM,49ers,...,0,-0.781831,0.623490,2,0.587785,-0.809017,False,True,False,False
129,2009,10,San Francisco 49ers,Chicago Bears,San Francisco 49ers,NaN,3,November 12,8:20PM,49ers,...,5,0.433884,-0.900969,2,0.587785,-0.809017,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2680,2019,8,Minnesota Vikings,Washington Redskins,Minnesota Vikings,NaN,3,October 24,8:20PM,Vikings,...,2,0.433884,-0.900969,1,0.951057,0.309017,False,False,True,False
2721,2019,11,Minnesota Vikings,Denver Broncos,Minnesota Vikings,NaN,6,November 17,1:00PM,Vikings,...,1,-0.781831,0.623490,2,0.587785,-0.809017,True,False,False,False
2767,2019,14,Minnesota Vikings,Detroit Lions,Minnesota Vikings,NaN,6,December 8,1:00PM,Vikings,...,2,-0.781831,0.623490,3,-0.587785,-0.809017,True,False,False,False
2799,2019,16,Minnesota Vikings,Green Bay Packers,Green Bay Packers,NaN,0,December 23,8:15PM,Vikings,...,3,0.000000,1.000000,3,-0.587785,-0.809017,False,False,True,False


In [142]:
# Calculate average attendance for each team and season
average_attendance_per_season = games_df.groupby(['year', 'home_team_name'])['weekly_attendance'].mean().reset_index()

# Rename and show average attendance per season
average_attendance_per_season.rename(columns={'weekly_attendance': 'avg_season_attendance'}, inplace=True)

# Lag avg_season_attendance by one year for each team
average_attendance_per_season['avg_season_attendance_prev'] = (
    average_attendance_per_season.groupby('home_team_name')['avg_season_attendance'].shift(1)
)

# Merge lagged average attendance into games_df using home_team and year
games_df = games_df.merge(
    average_attendance_per_season[['year', 'home_team_name', 'avg_season_attendance_prev']],
    left_on=['year', 'home_team_name'],
    right_on=['year', 'home_team_name'],
    how='left'
)

In [143]:
games_df

,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,...,yds_loss,turnovers_loss,home_team_name,home_team_city,away_team_name,away_team_city,weekly_attendance,weekly_attendance_ratio,prev_game_attendance,avg_season_attendance_prev
0,2009,2,San Francisco 49ers,Seattle Seahawks,San Francisco 49ers,NaN,Sun,September 20,4:05PM,23,...,283,1,49ers,San Francisco,Seahawks,Seattle,69732.0,0.764823,NaN,NaN
1,2009,4,San Francisco 49ers,St. Louis Rams,San Francisco 49ers,NaN,Sun,October 4,4:15PM,35,...,177,3,49ers,San Francisco,Rams,St. Louis,69732.0,0.764823,69732.0,NaN
2,2009,5,San Francisco 49ers,Atlanta Falcons,Atlanta Falcons,NaN,Sun,October 11,4:05PM,45,...,279,3,49ers,San Francisco,Falcons,Atlanta,69732.0,0.764823,69732.0,NaN
3,2009,9,San Francisco 49ers,Tennessee Titans,Tennessee Titans,NaN,Sun,November 8,4:15PM,34,...,358,4,49ers,San Francisco,Titans,Tennessee,69732.0,0.764823,69732.0,NaN
4,2009,10,San Francisco 49ers,Chicago Bears,San Francisco 49ers,NaN,Thu,November 12,8:20PM,10,...,350,5,49ers,San Francisco,Bears,Chicago,69732.0,0.764823,69732.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2019,8,Minnesota Vikings,Washington Redskins,Minnesota Vikings,NaN,Thu,October 24,8:20PM,19,...,216,2,Vikings,Minnesota,Redskins,Washington,66776.0,0.732289,66837.0,66811.375
2812,2019,11,Minnesota Vikings,Denver Broncos,Minnesota Vikings,NaN,Sun,November 17,1:00PM,27,...,394,1,Vikings,Minnesota,Broncos,Denver,66883.0,0.733463,66776.0,66811.375
2813,2019,14,Minnesota Vikings,Detroit Lions,Minnesota Vikings,NaN,Sun,December 8,1:00PM,20,...,231,2,Vikings,Minnesota,Lions,Detroit,66776.0,0.732289,66883.0,66811.375
2814,2019,16,Minnesota Vikings,Green Bay Packers,Green Bay Packers,NaN,Mon,December 23,8:15PM,23,...,139,1,Vikings,Minnesota,Packers,Green Bay,67157.0,0.736468,66776.0,66811.375


In [28]:
# Calculate average attendance for each team and season
average_attendance_per_season = attendance_df.groupby(['year', 'team_name'])['weekly_attendance'].mean().reset_index()

# Rename and show average attendance per season
average_attendance_per_season.rename(columns={'weekly_attendance': 'avg_season_attendance'}, inplace=True)

# Lag avg_season_attendance by one year for each team
average_attendance_per_season['avg_season_attendance_prev'] = (
    average_attendance_per_season.groupby('team_name')['avg_season_attendance'].shift(1)
)

# Merge lagged average attendance into games_df using home_team and year
games_df = games_df.merge(
    average_attendance_per_season[['year', 'team_name', 'avg_season_attendance_prev']],
    left_on=['year', 'home_team_name'],
    right_on=['year', 'team_name'],
    how='left'
).drop(columns=['team_name'])

# Last year performance

In [148]:
prev_year_winners = standings_df[standings_df['sb_winner'] == 'Won Superbowl'][["year", "team_name"]].copy()
prev_year_winners["year"] += 1  # Shift the year forward so we can match it to the next season

# Merge it with games_df on year and home_team
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "home_team_name"],
    right_on=["year", "team_name"]
)

# If the merge was successful, it means the home team was the previous year's Super Bowl winner
games_df["home_team_superbowl_winner_last_season"] = games_df["team_name"].notna()

# Drop the extra columns from the merge
games_df = games_df.drop(columns="team_name")

# Check if the away team is the previous year's Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)

# If the merge was successful, it means the away team was the previous year's Super Bowl winner
games_df["away_team_superbowl_winner_last_season"] = games_df["team_name"].notna()

games_df = games_df.drop(columns="team_name")

In [149]:
# Filter teams that made the playoffs (but not necessarily won the Super Bowl)
playoffs_finishers = standings_df[standings_df['playoffs'] == 'Playoffs'][["year", "team_name"]].copy()
playoffs_finishers["year"] += 1  # Shift to next season for matching

# Super Bowl winner
prev_year_winners = standings_df[standings_df['sb_winner'] == 'Won Superbowl'][["year", "team_name"]].copy()
prev_year_winners["year"] += 1

# Merge for home team: Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "home_team"],
    right_on=["year", "team_name"]
)
games_df["home_team_superbowl_winner_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for away team: Super Bowl winner
games_df = games_df.merge(
    prev_year_winners,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)
games_df["away_team_superbowl_winner_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for home team: Playoff participant
games_df = games_df.merge(
    playoffs_finishers,
    how="left",
    left_on=["year", "home_team"],
    right_on=["year", "team_name"]
)
games_df["home_team_playoffs_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

# Merge for away team: Playoff participant
games_df = games_df.merge(
    playoffs_finishers,
    how="left",
    left_on=["year", "away_team_name"],
    right_on=["year", "team_name"]
)
games_df["away_team_playoffs_last_season"] = games_df["team_name"].notna()
games_df = games_df.drop(columns="team_name")

In [150]:
# Ensure the data is sorted chronologically for correct rolling computation
games_df = games_df.sort_values(by=["year", "week"]).reset_index(drop=True)

# Create helper columns for win flags
games_df["home_win"] = (games_df["winner"] == games_df["home_team"]).astype(int)
games_df["away_win"] = (games_df["winner"] == games_df["away_team"]).astype(int)

# Build a long format DataFrame with one row per team per game
home_games = games_df[["year", "week", "home_team", "home_win"]].copy()
home_games.columns = ["year", "week", "team", "win"]

away_games = games_df[["year", "week", "away_team", "away_win"]].copy()
away_games.columns = ["year", "week", "team", "win"]

team_games = pd.concat([home_games, away_games]).sort_values(by=["team", "year", "week"]).reset_index(drop=True)

# Compute rolling win count over the last 3 games for each team
team_games["rolling_win_count_3wk"] = (
    team_games.groupby("team")["win"]
    .transform(lambda x: x.rolling(window=3, min_periods=3).sum().shift(1))  # shift(1) to exclude current week
)

# Fill NaNs with the last known value from previous year for the same team
team_games["rolling_win_count_3wk"] = (
    team_games.groupby("team")["rolling_win_count_3wk"]
    .transform(lambda x: x.ffill())
)

# Merge back into main DataFrame for home and away teams
# Home
home_rolling = team_games[["year", "week", "team", "rolling_win_count_3wk"]].copy()
home_rolling.columns = ["year", "week", "home_team", "home_team_wins_last_3"]

games_df = games_df.merge(home_rolling, on=["year", "week", "home_team"], how="left")

# Away
away_rolling = team_games[["year", "week", "team", "rolling_win_count_3wk"]].copy()
away_rolling.columns = ["year", "week", "away_team", "away_team_wins_last_3"]

games_df = games_df.merge(away_rolling, on=["year", "week", "away_team"], how="left")

games_df

,year,week,home_team,away_team,winner,tie,day,date,time,pts_win,...,prev_game_attendance,avg_season_attendance_prev,home_team_superbowl_winner_last_season,away_team_superbowl_winner_last_season,home_team_playoffs_last_season,away_team_playoffs_last_season,home_win,away_win,home_team_wins_last_3,away_team_wins_last_3
0,2009,1,Cincinnati Bengals,Denver Broncos,Denver Broncos,NaN,Sun,September 13,1:02PM,12,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
1,2009,1,Cleveland Browns,Minnesota Vikings,Minnesota Vikings,NaN,Sun,September 13,1:03PM,34,...,NaN,NaN,False,False,False,True,0,1,NaN,NaN
2,2009,1,Tampa Bay Buccaneers,Dallas Cowboys,Dallas Cowboys,NaN,Sun,September 13,1:03PM,34,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
3,2009,1,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,NaN,Sun,September 13,1:15PM,20,...,NaN,NaN,False,False,False,False,0,1,NaN,NaN
4,2009,1,Indianapolis Colts,Jacksonville Jaguars,Indianapolis Colts,NaN,Sun,September 13,1:02PM,14,...,NaN,NaN,False,False,False,False,1,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2019,17,Los Angeles Rams,Arizona Cardinals,Los Angeles Rams,NaN,Sun,December 29,4:25PM,31,...,71501.0,72429.875,False,False,False,False,1,0,1.0,2.0
2812,2019,17,Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,NaN,Sun,December 29,4:25PM,28,...,70545.0,70431.375,False,False,False,False,1,0,3.0,1.0
2813,2019,17,Seattle Seahawks,San Francisco 49ers,San Francisco 49ers,NaN,Sun,December 29,8:20PM,26,...,69022.0,69001.125,False,False,False,False,0,1,1.0,2.0
2814,2019,17,Houston Texans,Tennessee Titans,Tennessee Titans,NaN,Sun,December 29,4:25PM,35,...,71769.0,71804.875,False,False,False,False,0,1,2.0,1.0


# Redifining pts in terms of home and away 

In [151]:
# Safer and clearer naming
games_df["pts_home"] = games_df.apply(lambda row: row["pts_win"] if row["winner"] == row["home_team"] else row["pts_loss"], axis=1)
games_df["pts_away"] = games_df.apply(lambda row: row["pts_win"] if row["winner"] == row["away_team"] else row["pts_loss"], axis=1)

games_df["yds_home"] = games_df.apply(lambda row: row["yds_win"] if row["winner"] == row["home_team"] else row["yds_loss"], axis=1)
games_df["yds_away"] = games_df.apply(lambda row: row["yds_win"] if row["winner"] == row["away_team"] else row["yds_loss"], axis=1)

games_df["turnovers_home"] = games_df.apply(lambda row: row["turnovers_win"] if row["winner"] == row["home_team"] else row["turnovers_loss"], axis=1)
games_df["turnovers_away"] = games_df.apply(lambda row: row["turnovers_win"] if row["winner"] == row["away_team"] else row["turnovers_loss"], axis=1)

# Drop the original columns that are no longer needed
games_df = games_df.drop(columns=["pts_win", "pts_loss", "yds_win", "yds_loss", "turnovers_win", "turnovers_loss"])

# Standarize with previous year values

In [25]:
# # Combine all stats: Points, Yards, Turnovers
# # Home stats
# home_stats = games_df[["year", "home_team", "pts_win", "yds_win", "turnovers_win"]].rename(
#     columns={
#         "home_team": "team",
#         "pts_win": "points",
#         "yds_win": "yds",
#         "turnovers_win": "turnovers"
#     }
# )

# # Away stats
# away_stats = games_df[["year", "away_team", "pts_loss", "yds_loss", "turnovers_loss"]].rename(
#     columns={
#         "away_team": "team",
#         "pts_loss": "points",
#         "yds_loss": "yds",
#         "turnovers_loss": "turnovers"
#     }
# )

# # Combine for per-team/year averages
# all_stats = pd.concat([home_stats, away_stats])
# avg_stats = all_stats.groupby(["team", "year"])[["points", "yds", "turnovers"]].mean().reset_index()

# # Shift year so previous season stats apply to current games
# avg_stats["year"] += 1

# # Merge previous season averages into games_df for both home and away teams
# for prefix, team_col in [("home", "home_team"), ("away", "away_team")]:
#     games_df = games_df.merge(
#         avg_stats.rename(columns={
#             "team": team_col,
#             "points": f"{prefix}_avg_prev_pts",
#             "yds": f"{prefix}_avg_prev_yds",
#             "turnovers": f"{prefix}_avg_prev_turnovers"
#         }),
#         on=[team_col, "year"],
#         how="left"
#     )

# # ----- Standardize all stats -----
# games_df["standardized_pts_win"] = games_df["pts_win"] / games_df["home_avg_prev_pts"]
# games_df["standardized_pts_loss"] = games_df["pts_loss"] / games_df["away_avg_prev_pts"]

# games_df["standardized_yds_win"] = games_df["yds_win"] / games_df["home_avg_prev_yds"]
# games_df["standardized_yds_loss"] = games_df["yds_loss"] / games_df["away_avg_prev_yds"]

# games_df["standardized_turnovers_win"] = games_df["turnovers_win"] / games_df["home_avg_prev_turnovers"]
# games_df["standardized_turnovers_loss"] = games_df["turnovers_loss"] / games_df["away_avg_prev_turnovers"]

# Z-score

In [152]:
# Prepare home team stats
home_stats = games_df[["year", "home_team_name", "pts_home", "yds_home", "turnovers_home"]].rename(
    columns={
        "home_team_name": "team",
        "pts_home": "points",
        "yds_home": "yds",
        "turnovers_home": "turnovers"
    }
)

# Prepare away team stats
away_stats = games_df[["year", "away_team_name", "pts_away", "yds_away", "turnovers_away"]].rename(
    columns={
        "away_team_name": "team",
        "pts_away": "points",
        "yds_away": "yds",
        "turnovers_away": "turnovers"
    }
)

# Combine into long format: one row per team-game
all_stats = pd.concat([home_stats, away_stats], ignore_index=True)

# Compute per-team/year means and stds
team_year_stats = (
    all_stats.groupby(["team", "year"])[["points", "yds", "turnovers"]]
    .agg(["mean", "std"])
    .reset_index()
)

# Flatten MultiIndex columns
team_year_stats.columns = [
    "team", "year",
    "points_mean", "points_std",
    "yds_mean", "yds_std",
    "turnovers_mean", "turnovers_std"
]

# Shift to apply previous year's stats to current year's games
team_year_stats["year"] += 1

# Merge into games_df for both home and away teams
for prefix, team_col in [("home", "home_team_name"), ("away", "away_team_name")]:
    stats = team_year_stats.rename(columns={
        "team": team_col,
        "points_mean": f"{prefix}_prev_pts_mean",
        "points_std": f"{prefix}_prev_pts_std",
        "yds_mean": f"{prefix}_prev_yds_mean",
        "yds_std": f"{prefix}_prev_yds_std",
        "turnovers_mean": f"{prefix}_prev_turnovers_mean",
        "turnovers_std": f"{prefix}_prev_turnovers_std"
    })
    games_df = games_df.merge(stats, on=[team_col, "year"], how="left")


# Home Z-scores
games_df["zscore_pts_home"] = (games_df["pts_home"] - games_df["home_prev_pts_mean"]) / games_df["home_prev_pts_std"]
games_df["zscore_yds_home"] = (games_df["yds_home"] - games_df["home_prev_yds_mean"]) / games_df["home_prev_yds_std"]
games_df["zscore_turnovers_home"] = (games_df["turnovers_home"] - games_df["home_prev_turnovers_mean"]) / games_df["home_prev_turnovers_std"]

# Away Z-scores
games_df["zscore_pts_away"] = (games_df["pts_away"] - games_df["away_prev_pts_mean"]) / games_df["away_prev_pts_std"]
games_df["zscore_yds_away"] = (games_df["yds_away"] - games_df["away_prev_yds_mean"]) / games_df["away_prev_yds_std"]
games_df["zscore_turnovers_away"] = (games_df["turnovers_away"] - games_df["away_prev_turnovers_mean"]) / games_df["away_prev_turnovers_std"]

In [27]:
games_df.isna().sum()

year                                         0
week                                         0
home_team                                    0
away_team                                    0
winner                                       0
tie                                       2808
day                                          0
date                                         0
time                                         0
home_team_name                               0
home_team_city                               0
away_team_name                               0
away_team_city                               0
weekly_attendance                            0
weekly_attendance_ratio                      0
prev_game_attendance                        32
avg_season_attendance_prev                 256
home_team_superbowl_winner_last_season       0
away_team_superbowl_winner_last_season       0
home_team_playoffs_last_season               0
away_team_playoffs_last_season               0
home_win     

# Temporal features processing

In [163]:
games_df_proc = pd.read_csv('../processed_data/games.csv')

In [153]:
# games_df_proc = pd.read_csv('../processed_data/games.csv')
games_df_proc = games_df.copy()

In [164]:
day_of_the_week = {'Mon':0, 'Tue':1, 'Wed':2, 'Thu':3, 'Fri':4, 'Sat':5, 'Sun':6}
games_df_proc['day'] = games_df_proc['day'].map(day_of_the_week)

In [165]:

N = 7
days = games_df_proc['day'].values
# Compute sine and cosine components
days_sin = np.sin(2 * np.pi * days / N)
days_cos = np.cos(2 * np.pi * days / N)
games_df_proc['day_sin'] = days_sin
games_df_proc['day_cos'] = days_cos

In [166]:
months_year = {'September': 0, 'October': 1, 'November': 2, 'December': 3, 'January': 4}
df = games_df_proc['date'].str.split(' ', expand=True)
games_df_proc['month'] = df[0].map(months_year)

In [167]:
N = 5
days = games_df_proc['month'].values
# Compute sine and cosine components
days_sin = np.sin(2 * np.pi * days / N)
days_cos = np.cos(2 * np.pi * days / N)
games_df_proc['month_sin'] = days_sin
games_df_proc['month_cos'] = days_cos

In [168]:
hour = games_df_proc['time'].str.split(':', expand=True)
hour.rename(columns={0: 'hour', 1: 'minute'}, inplace=True)
hour.loc[hour['hour']==12, 'hour'] = 0
hour['hour'] = hour['hour'].astype(int)


In [169]:
games_df_proc['noon'] = hour['hour']<2
games_df_proc['mid_afternoon'] = (2<=hour['hour']) & (hour['hour']<7)
games_df_proc['late_afternoon'] = (7<=hour['hour']) & (hour['hour']<9)
games_df_proc['evening'] = (9<=hour['hour'])

In [125]:
games_df_proc.columns

Index(['year', 'week', 'home_team_name', 'away_team_name', 'day', 'date',
       'time', 'weekly_attendance', 'weekly_attendance_ratio',
       'prev_game_attendance', 'avg_season_attendance_prev',
       'home_team_superbowl_winner_last_season',
       'away_team_superbowl_winner_last_season',
       'home_team_playoffs_last_season', 'away_team_playoffs_last_season',
       'home_team_wins_last_3', 'away_team_wins_last_3', 'pts_home',
       'pts_away', 'yds_home', 'yds_away', 'turnovers_home', 'turnovers_away',
       'zscore_pts_home', 'zscore_yds_home', 'zscore_turnovers_home',
       'zscore_pts_away', 'zscore_yds_away', 'zscore_turnovers_away',
       'day_sin', 'day_cos', 'month', 'month_sin', 'month_cos', 'noon',
       'mid_afternoon', 'late_afternoon', 'evening'],
      dtype='object')

In [173]:
games_df

,year,week,home_team,away_team,winner,tie,day,date,time,home_team_name,...,away_prev_yds_mean,away_prev_yds_std,away_prev_turnovers_mean,away_prev_turnovers_std,zscore_pts_home,zscore_yds_home,zscore_turnovers_home,zscore_pts_away,zscore_yds_away,zscore_turnovers_away
0,2009,1,Cincinnati Bengals,Denver Broncos,Denver Broncos,NaN,Sun,September 13,1:02PM,Bengals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,1,Cleveland Browns,Minnesota Vikings,Minnesota Vikings,NaN,Sun,September 13,1:03PM,Browns,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2009,1,Tampa Bay Buccaneers,Dallas Cowboys,Dallas Cowboys,NaN,Sun,September 13,1:03PM,Buccaneers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2009,1,Arizona Cardinals,San Francisco 49ers,San Francisco 49ers,NaN,Sun,September 13,1:15PM,Cardinals,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2009,1,Indianapolis Colts,Jacksonville Jaguars,Indianapolis Colts,NaN,Sun,September 13,1:02PM,Colts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811,2019,17,Los Angeles Rams,Arizona Cardinals,Los Angeles Rams,NaN,Sun,December 29,4:25PM,Rams,...,241.5625,51.817589,1.750,1.390444,-0.181839,0.035264,-0.970939,1.317844,2.922512,2.337384
2812,2019,17,Baltimore Ravens,Pittsburgh Steelers,Baltimore Ravens,NaN,Sun,December 29,4:25PM,Ravens,...,403.3125,67.978152,1.625,1.627882,0.456807,-1.383681,0.805629,-1.620293,-3.461590,0.230361
2813,2019,17,Seattle Seahawks,San Francisco 49ers,San Francisco 49ers,NaN,Sun,December 29,8:20PM,Seahawks,...,360.5625,62.283191,2.000,1.673320,-0.820313,-0.095471,-0.787323,0.551286,0.601085,-1.195229
2814,2019,17,Houston Texans,Tennessee Titans,Tennessee Titans,NaN,Sun,December 29,4:25PM,Texans,...,312.6250,84.608806,1.125,1.147461,-1.490416,-0.788735,0.000000,1.773927,1.824574,-0.980426


In [172]:
games_df_proc

,year,home_team_name,weekly_attendance,weekly_attendance_ratio,prev_game_attendance,avg_season_attendance_prev,home_team_superbowl_winner_last_season,away_team_superbowl_winner_last_season,home_team_playoffs_last_season,away_team_playoffs_last_season,...,zscore_yds_away,zscore_turnovers_away,day_sin,day_cos,month_sin,month_cos,noon,mid_afternoon,late_afternoon,evening
0,2010,Bears,62080.0,0.685558,62479.0,64444.6250,False,False,False,False,...,-1.711130,0.268122,-0.781831,0.62349,0.000000,1.000000,True,False,False,False
1,2010,Bills,69295.0,0.766156,69848.0,67546.5000,False,False,False,False,...,-0.509086,-1.196211,-0.781831,0.62349,0.000000,1.000000,True,False,False,False
2,2010,Buccaneers,47211.0,0.507094,62578.0,68354.3125,False,False,False,False,...,0.812194,0.696191,-0.781831,0.62349,0.000000,1.000000,True,False,False,False
3,2010,Patriots,68756.0,0.738883,68756.0,71125.0000,False,False,False,True,...,1.305159,0.346325,-0.781831,0.62349,0.000000,1.000000,True,False,False,False
4,2010,Steelers,63609.0,0.665401,64068.0,65711.6250,False,False,False,False,...,-0.631865,-0.514384,-0.781831,0.62349,0.000000,1.000000,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,2019,Jaguars,58472.0,0.644199,57866.0,69368.6875,False,False,False,True,...,-1.463446,0.412861,-0.781831,0.62349,-0.587785,-0.809017,False,True,False,False
2556,2019,Rams,68665.0,0.759266,71501.0,68900.5625,False,False,False,False,...,2.922512,2.337384,-0.781831,0.62349,-0.587785,-0.809017,False,True,False,False
2557,2019,Ravens,70695.0,0.759704,70545.0,65836.4375,False,False,False,False,...,-3.461590,0.230361,-0.781831,0.62349,-0.587785,-0.809017,False,True,False,False
2558,2019,Texans,71794.0,0.787569,71769.0,69530.0000,False,False,False,False,...,1.824574,-0.980426,-0.781831,0.62349,-0.587785,-0.809017,False,True,False,False


In [175]:
games_df_proc.drop(columns=["avg_season_attendance_prev"], inplace=True)

In [179]:
games_df_proc = games_df_proc.merge(
    average_attendance_per_season[['year', 'home_team_name', 'avg_season_attendance_prev']],
    left_on=['year', 'home_team_name'],
    right_on=['year', 'home_team_name'],
    how='left'
)

In [ ]:
games_df

In [180]:
games_df_proc.drop(columns=['week', 'day', 'month', 'date', 'time', 'away_team_name'], inplace=True)

KeyError: "['week', 'day', 'month', 'date', 'time', 'away_team_name'] not found in axis"

In [171]:
print(games_df_proc.shape)
print(games_df.shape)

(2560, 32)
(2816, 49)


In [181]:
games_df_proc.to_csv('../processed_data/games_filtered.csv', index=False)

# Drop 2009 year which was used just for having the first year as no missings, and redundant cols

In [28]:
games_df.columns

Index(['year', 'week', 'home_team', 'away_team', 'winner', 'tie', 'day',
       'date', 'time', 'home_team_name', 'home_team_city', 'away_team_name',
       'away_team_city', 'weekly_attendance', 'weekly_attendance_ratio',
       'prev_game_attendance', 'avg_season_attendance_prev',
       'home_team_superbowl_winner_last_season',
       'away_team_superbowl_winner_last_season',
       'home_team_playoffs_last_season', 'away_team_playoffs_last_season',
       'home_win', 'away_win', 'home_team_wins_last_3',
       'away_team_wins_last_3', 'pts_home', 'pts_away', 'yds_home', 'yds_away',
       'turnovers_home', 'turnovers_away', 'home_prev_pts_mean',
       'home_prev_pts_std', 'home_prev_yds_mean', 'home_prev_yds_std',
       'home_prev_turnovers_mean', 'home_prev_turnovers_std',
       'away_prev_pts_mean', 'away_prev_pts_std', 'away_prev_yds_mean',
       'away_prev_yds_std', 'away_prev_turnovers_mean',
       'away_prev_turnovers_std', 'zscore_pts_home', 'zscore_yds_home',
       'z

In [29]:
columns_to_select = ['year', 'week', 'home_team_name', 'away_team_name','day','date', 'time', 
    'weekly_attendance', 'weekly_attendance_ratio', 'prev_game_attendance', 'avg_season_attendance_prev', 'home_team_superbowl_winner_last_season',
    'away_team_superbowl_winner_last_season', 'home_team_playoffs_last_season', 
    'away_team_playoffs_last_season','home_team_wins_last_3', 'away_team_wins_last_3',
    'pts_home', 'pts_away', 'yds_home', 'yds_away', 'turnovers_home', 'turnovers_away', # I keep it for cheching this is not modified
    'zscore_pts_home', 'zscore_yds_home','zscore_turnovers_home', 'zscore_pts_away', 'zscore_yds_away','zscore_turnovers_away'
    ]

games_df = games_df[games_df["year"]>2009][columns_to_select].copy().sort_values(by=['year', 'week', 'date', 'time'])

In [30]:
# Save games_df to the process folder as a CSV file
process_folder = "./../processed_data"
os.makedirs(process_folder, exist_ok=True)
games_df.to_csv(os.path.join(process_folder, "games.csv"), index=False)

# Merging weather data (New York Giants)

In [31]:
team_name = 'Giants'
nickname = team_name.split(' ')[-1].lower()

weather_data_path = './../weather_data'
weather_df = pd.read_csv(weather_data_path+'/export_'+nickname+'.csv')

games_df['full_date'] = pd.to_datetime(games_df['date'] + ' ' + games_df['year'].astype(str), format='%B %d %Y')

df_pats = games_df.loc[games_df['home_team_name'] == team_name]

weather_df.rename(columns={'date': 'full_date'}, inplace=True)
weather_df['full_date'] = pd.to_datetime(weather_df['full_date'])

df_pats = df_pats.merge(weather_df, on=['full_date'], how='left')

process_folder = "./../datasets_w_weather_data"
os.makedirs(process_folder, exist_ok=True)
df_pats.to_csv(os.path.join(process_folder, f"{team_name}.csv"), index=False)